In [ ]:
%reload_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv
%matplotlib inline

In [ ]:
import pandas as pd
import os
import sys
import json
from datetime import datetime


In [ ]:
sys.path.insert(0,os.path.dirname('../src/'))

In [ ]:
E1_template = pd.read_csv("../data/E1_template_from_RHEA.csv")

In [ ]:
E1_template.head(2)

In [ ]:
E1_csv_output = pd.DataFrame(columns = E1_template.columns)

In [ ]:
E1_csv_output.columns

In [ ]:
with open('../data/aoi.json', 'r') as f:
    aoi_file = json.load(f)

interest = 'Final'

In [ ]:
def get_color_code(date_time_str):
    dt_obj = datetime.strptime(date_time_str, '%Y-%m-%d')
    if dt_obj.year == 2020:
        return "RED"
    else:
        return "GREEN"
    
sitename_to_countrycode = {
        "Sines" : "PT",
        "Vigo" : "ES",
        "Antwerp" : "BE",
        "Budapest" : "HU" ,
        "Rotterdam" : "NL",
        "Kiel Canal": "DE",
        "Grand Harbour" : "MT",
        "Dublin" : "IE",
        "Tallin-Helsinki": "FI",
        "Limassol" : "CY",
        "Athen Pireaus" : "GR",
        "Bosporus" : "TR",
        "Dardanelles" : "TR",
        "Gibraltar" :"GB",
        "Dover" : "GB",
        "Corfu" : "GR",
        "Messina": "IT",
        "Oresund": "SE",
        "Venice" : "IT",
        "St Tropez" :"FR",
        "Split" : "HR"
    }
sitename_to_city = {
        "Sines" : "Sines",
        "Vigo" : "Vigo",
        "Antwerp" : "Antwerp",
        "Budapest" : "Budapest" ,
        "Rotterdam" : "Rotterdam",
        "Kiel Canal": "Kiel",
        "Grand Harbour" : "Valleta",
        "Dublin" : "Dublin",
        "Tallin-Helsinki": "Helsinki",
        "Limassol" : "Limassol",
        "Athen Pireaus" : "Athen",
        "Bosporus" : "Bosporus",
        "Dardanelles" : "Dardanelles",
        "Gibraltar" :"Gibraltar",
        "Dover" : "Dover",
        "Corfu" : "Corfu",
        "Messina": "Messina",
        "Oresund": "Oresund",
        "Venice" : "Venice",
        "St Tropez" :"St Tropez",
        "Split" : "Split"
    }
  

In [ ]:
radius=7000
version="0.1.1"
for sitename, lat_lons in aoi_file[interest].items():
    for lat_lon in lat_lons:
        output_file = '{}_lat={}_lon={}_r={}_v={}.json'.format(sitename.lower(), lat_lon[0], lat_lon[1], radius, version)
        output_full_path = os.path.join("../data/outputs", output_file)
        if os.path.isfile(output_full_path):
            f = open(output_full_path)
            output_obj = json.load(f)
            for time, count in output_obj.items():
#                 print(f"time: {time}, count: {count}")
                AOI = f"{lat_lon[0]},{lat_lon[1]}"
                Country = sitename_to_countrycode[sitename]
                City = sitename_to_city[sitename]
                Site_name = sitename
                Description = ""
                Method = "Object Counting"
                EO_sensor = "Sentinel-2"
                Input_data = "Sentinel 2 L1C"
                Indicator_code = "E1"
                Reference_description = f"Number of ships in {2 * radius / 1000} km x {2 * radius / 1000} km area"
                
                if datetime.strptime(time, '%Y-%m-%d').year == 2020:
                    Time = time
                    Measurement_value = count
                    Reference_time = ""
                    Reference_value = ""
                else:
                    Time = ""
                    Measurement_value = ""
                    Reference_time = time
                    Reference_value = count
                
                Rule = ""
                Indicator_Value = ""
                Sub_AOI = ""
                Y_axis = "Number of ships inferred"
                Indicator_Name = "Changes in Ships traffic"
                Color_code = get_color_code(time)
                Data_Provider = ""
                AOI_ID = ""
                Update_Frequency = "Weekly"
                row = {'AOI':AOI, 'Country':Country, 'Region':"", 'City':City, 'Site Name':Site_name, 'Description':Description,
                       'Method':Method, 'EO Sensor':EO_sensor, 'Input Data':Input_data, 'Indicator code':Indicator_code, 'Time':Time,
                       'Measurement Value':Measurement_value, 'Reference Description':Reference_description, 'Reference time': Reference_time,
                       'Reference value':Reference_value, 'Rule':Rule, 'Indicator Value':Indicator_Value, 'Sub-AOI':Sub_AOI, 'Y axis':Y_axis,
                       'Indicator Name':Indicator_Name, 'Color code':Color_code, 'Data Provider':Data_Provider, 'AOI_ID':AOI_ID,
                       'Update Frequency':Update_Frequency}
                E1_csv_output = E1_csv_output.append(row, ignore_index=True)
            

In [ ]:
E1_csv_output.head(3)

In [ ]:
E1_csv_output.to_csv(f"../data/E1_{datetime.now()}.csv")